# Explore Training Curves

Modified by Jun, based on Patrick Coady's code: https://github.com/pat-coady/trpo

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set()
sns.set_style("whitegrid")
%matplotlib inline
plt.rcParams["figure.figsize"] = (6,5)

In [3]:
def find_moving_avg(data, window_size):
    quantile1 = np.zeros(len(data)-window_size)
    quantile2 = np.zeros(len(data)-window_size)
    avg = np.zeros(len(data)-window_size)
    for i in range(len(data)-window_size):
        avg[i] = np.mean(data[i:i+window_size])
        quantile1[i] = np.quantile(data[i:i+window_size], 0.25)
        quantile2[i] = np.quantile(data[i:i+window_size], 0.75)
    return avg, quantile1, quantile2

def df_plot(dfs, x, ys, isTaxi, isChain, isCliff, ylim=None, xlim=None, legend_loc='best'):
    """ Plot y vs. x curves from pandas dataframe(s)

    Args:
        dfs: list of pandas dataframes
        x: str column label for x variable
        ys: list of str column labels for y variable(s)
        ylim: tuple to override automatic y-axis limits
        xlim: tuple to override automatic x-axis limits
        legend_loc: str to override automatic legend placement:
            'upper left', 'lower left', 'lower right' , 'right' ,
            'center left', 'center right', 'lower center',
            'upper center', and 'center'
    """
    if ylim is not None:
        plt.ylim(ylim)
    if xlim is not None:
        plt.xlim(xlim)
    for df, name in dfs:
        if '_' in name:
            name = name.split('_')[1]
            if 'KL' in name:
                name = 'DRTRPO-KL'
            else:
                name = 'DRTRPO-Wass'
        for y in ys:
            x_val = df[x]
            y_val = df[y]
            ws = 10
            if isTaxi == True:
                timesteps_per_eps = 70
                plt.title('Taxi', fontsize = 26)
            if isChain == True:
                timesteps_per_eps = 1000
                ws = 30
                plt.title('Chain', fontsize = 26)
            if isCliff == True:
                timesteps_per_eps = 50
                plt.title('Cliff', fontsize = 26)
            moving_average = find_moving_avg(y_val, ws)
            plt.plot(x_val[0:len(x_val)-ws]*timesteps_per_eps/100000, moving_average[0], linewidth = 2.5, label=name)
            plt.fill_between(x_val[0:len(x_val)-ws]*timesteps_per_eps/100000, moving_average[1], moving_average[2], alpha=0.3)
    plt.xticks(fontsize=22)
    plt.yticks(fontsize=22)
    plt.axis((None,None,None,None))
    plt.xlabel('            Timesteps       '+ r'$\times 10^5$',fontsize=22)
    plt.ylabel('Average Return',fontsize=22)

In [12]:
isTaxi = True
isChain = False
isCliff = False


# ENTER LIST OF LOG FILENAMES HERE:
if isTaxi:
    filepaths = ['log-files/Taxi-v3_DR-KL_Batch=60/Feb-22_01:56:50/log.csv', 'log-files/Taxi-v3_DR-Wass_Batch=60/Feb-22_01:40:39/log.csv']
if isChain:
    filepaths = ['log-files/NChain-v0_DR-KL_Batch=1/Feb-19_01:46:23/log.csv', 'log-files/NChain-v0_DR-Wass_Batch=1/Feb-20_07:48:50/log.csv']
if isCliff:
    filepaths = ['log-files/CliffWalking-v0_DR-KL_Batch=5/Feb-18_10:52:31/log.csv', 'log-files/CliffWalking-v0_DR-Wass_Batch=3/Feb-18_16:41:21/log.csv']

dataframes = []
names = []
for filepath in filepaths:
    names.append(filepath.split('/')[1])
    dataframes.append(pd.read_csv(filepath))
data = list(zip(dataframes, names))

# Plots

In [13]:
if isTaxi:
    df_plot(data, '_Episode', ['_AvgRewardSum'], isTaxi, isChain, isCliff, None, [0,3])
if isChain:
    df_plot(data, '_Episode', ['_AvgRewardSum'], isTaxi, isChain, isCliff, None, [0,8.0])
if isCliff:
    df_plot(data, '_Episode', ['_AvgRewardSum'], isTaxi, isChain, isCliff, None, [0,0.3])
    
window_size = 10

# read TRPO baseline results
if isTaxi:
    trpo_log_name = './tmp/taxi_monitor.csv'
if isChain:
    trpo_log_name = './tmp/chain_monitor.csv'
    window_size = 30
if isCliff:
    trpo_log_name = './tmp/cliff_monitor.csv'

df=pd.read_csv(trpo_log_name, sep=',', skiprows = 0, header= 1)
y_val = df['r']
moving_average = find_moving_avg(y_val, window_size)
trpo_ts_per_iter = 1024
plt.plot(np.arange(len(y_val)-window_size)*trpo_ts_per_iter/100000, moving_average[0], label='TRPO', linewidth = 2.5)
plt.fill_between(np.arange(len(y_val)-window_size)*trpo_ts_per_iter/100000, moving_average[1], moving_average[2], alpha=0.2)
plt.rcParams["legend.framealpha"] = 0.5
plt.legend(fontsize=18, loc='lower right')